In [1]:
import os
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DoubleType, TimestampType
from pyspark.sql.functions import col, to_date, concat, lit
os.environ["SPARK_HOME"] = "/home/hel/.local/lib/python3.10/site-packages/pyspark/"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("My Spark Application") \
    .getOrCreate()


24/05/03 13:08:34 WARN Utils: Your hostname, hel-ThinkPad-T14-Gen-3 resolves to a loopback address: 127.0.1.1; using 192.168.1.24 instead (on interface wlp2s0)
24/05/03 13:08:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 13:08:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import col, dayofweek,to_date, month, count, avg
from pyspark.sql import Window
from pyspark.sql.functions import row_number,   sum, when


# Load the CSV file into a DataFrame
csv_file_path_flightdelay = "./full_data_flightdelay.csv"  # Replace with the path to your CSV file


df_flightdelay = spark.read.option("delimiter", ",").option("header", "true").csv(csv_file_path_flightdelay)


# Read the CSV file using the manually defined schema
csv_file_path_weather = "./airport_weather_2019.csv"  # Replace with your file path
df_weather = spark.read.option("delimiter", ",").option("header", "true").csv(csv_file_path_weather)




In [4]:
print(df_weather.count())

38675


In [5]:
from pyspark.sql.functions import coalesce

# create new column for month and day_of_week values derived from date
df_day_column = df_weather.withColumn("DATE_NEW", to_date(col("DATE"), "M/d/yyyy"))
df_day_column = df_day_column.withColumn("DATE_NEW", coalesce(df_day_column["DATE_NEW"], to_date(df_day_column["DATE"], 'yyyy-MM-dd')))
    
df_day_column = df_day_column.withColumn("DAY_OF_WEEK", dayofweek(col("DATE_NEW").alias("DAY_OF_WEEK")))
df_day_column = df_day_column.withColumn("MONTH", month(col("DATE_NEW").alias("MONTH")))
#df_day_column = df_weather.withColumn("DAY_OF_WEEK", dayofweek(col("DATE").alias("DAY_OF_WEEK"))) 




#df_day_column.show(n=2)
df_day_column.createOrReplaceTempView("table1")
df_select = spark.sql("SELECT STATION, NAME,DAY_OF_WEEK,DATE, MONTH, AWND, PRCP, SNOW, SNWD, TAVG, TMAX, TMIN, WDF2 from table1")
#df_select.show(n=5)

grouped_df = df_select.groupBy("MONTH", "NAME").agg(
    avg("AWND").alias("AWND"),
    avg("PRCP").alias("PRCP"),
    avg("SNOW").alias("SNOW"),
    avg("SNWD").alias("SNWD"),
    avg("TAVG").alias("TAVG"),
    avg("TMAX").alias("TMAX"),
    avg("TMIN").alias("TMIN"),
    avg("WDF2").alias("WDF2")
).orderBy("NAME","MONTH")


grouped_df.show(n=20)

24/05/03 13:08:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+--------------------+------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+
|MONTH|                NAME|              AWND|                PRCP|               SNOW|               SNWD|              TAVG|              TMAX|              TMIN|              WDF2|
+-----+--------------------+------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+
|    1|ALBANY INTERNATIO...| 9.604516129032257| 0.13838709677419353| 0.6096774193548388| 2.5064516129032257|23.967741935483872| 31.93548387096774|14.709677419354838|231.29032258064515|
|    2|ALBANY INTERNATIO...| 8.803214285714287| 0.09571428571428572| 0.5035714285714284|              2.525|27.857142857142858|35.392857142857146|             19.75|233.57142857142858|
|    3|ALBANY INTERNATIO...| 9.698064516129032|0.045000000000000005|0.21290

In [6]:
from pyspark.sql.functions import lower, split, col, lit, monotonically_increasing_id


# Normalize joining columns
grouped_df = grouped_df.withColumn("normalized_name", lower(col("name")))
df_flightdelay = df_flightdelay.withColumn("normalized_name", lower(split(col("departing_airport"), " ").getItem(0)))

# Group by to investigate
grouped_df_nn = grouped_df.groupBy("normalized_name").agg(
    count('*').alias('count')
)

grouped_df_name = grouped_df.groupBy("NAME").agg(
    count('*').alias('count')
)

In [10]:

# Only unique values
grouped_df = grouped_df.select('name').distinct()
#print(grouped_df.count())
#print("Unique airports __names__ in flightdelay data: ",df_flightdelay.select('departing_airport').distinct().count())
print("Unique __departing airports__ in flightdelay data: ",df_flightdelay.select('DEPARTING_AIRPORT').distinct().count())
df_flightdelay.select('DEPARTING_AIRPORT').distinct().show(n=2)

Unique __departing airports__ in flightdelay data:  86


+-----------------+
|DEPARTING_AIRPORT|
+-----------------+
|  Eppley Airfield|
|  Kahului Airport|
+-----------------+
only showing top 2 rows



In [23]:

# Join dataframes grouped_df and df_flightdelay

#joined_df = df_flightdelay.alias('f').join(
#    grouped_df.alias('g'),
#    (col('g.month') == col('f.month')) & (col('g.normalized_name')).contains(col('f.normalized_name')), 'inner'
#)


# For 'grouped_df', transforming 'NAME' to lowercase and dropping duplicates based on the 'name' column
grouped_df_lower = grouped_df.select(lower(col("NAME")).alias("name")).dropDuplicates(['name'])

# For 'df_flightdelay', transforming 'DEPARTING_AIRPORT' to lowercase, casting it to string, and dropping duplicates based on the 'departing_airport' column
df_flightdelay_lower = df_flightdelay.select(lower(col("DEPARTING_AIRPORT")).alias("departing_airport")).dropDuplicates(['departing_airport'])



#join providing table that contain in the name column all distinct airports 
#from weather dataset and under departing_flight all distinc airports from delay dataset
print(grouped_df_lower.count())
print(df_flightdelay_lower.count())
result_df = df_flightdelay_lower.alias("flight").join(
    grouped_df_lower.alias("grouped"),
    (col("grouped.name").contains(col("flight.departing_airport"))),
    "inner"
).select(
    col("flight.departing_airport").alias("departing_airport"),
    col("grouped.name").alias("name")
)

result_df.show(n=2)




106


86


+--------------------+--------------------+
|   departing_airport|                name|
+--------------------+--------------------+
|memphis internati...|memphis internati...|
|portland internat...|portland internat...|
+--------------------+--------------------+
only showing top 2 rows



In [24]:
#modify dataframe such that df_result will contain the airports matched on join 
#and enhanced results will contain the df of unmatched airports for each dataset

# Identifying non-matched entries
non_matched_flight = df_flightdelay_lower.alias("flight").join(
    result_df.alias("result"),
    result_df.departing_airport == df_flightdelay_lower.departing_airport,
    "left_anti"
)

non_matched_grouped = grouped_df_lower.alias("grouped").join(
    result_df.alias("result"),
    result_df.name == grouped_df_lower.name,
    "left_anti"
)


# Add a unique ID to each DataFrame to facilitate the outer join
result_df = result_df.withColumn("id", monotonically_increasing_id())
non_matched_flight = non_matched_flight.withColumn("id", monotonically_increasing_id())
non_matched_grouped = non_matched_grouped.withColumn("id", monotonically_increasing_id())

# Perform the outer joins using the unique IDs, result_df is now composed of matched airports
enhanced_result_df = result_df.join(non_matched_flight, "id", "outer" ).join(non_matched_grouped, "id", "outer" )
enhanced_result_df = enhanced_result_df.drop("id")

# Show the enhanced DataFrame with additional columns
enhanced_result_df.printSchema()

# Select columns, get rid of duplicates
selected_columns = [col for col in enhanced_result_df.columns if col != 'name' and col != 'departing_airport'] + ['grouped.name'] + ['flight.departing_airport']

#will contain unmatched airports for each dataset
enhanced_result_df = enhanced_result_df.select(selected_columns)
enhanced_result_df.drop('name','departing_airport')
enhanced_result_df.show(n=2)


root
 |-- departing_airport: string (nullable = true)
 |-- name: string (nullable = true)
 |-- departing_airport: string (nullable = true)
 |-- name: string (nullable = true)



+--------------------+--------------------+
|                name|   departing_airport|
+--------------------+--------------------+
|albany internatio...|tucson international|
|albuquerque inter...|charleston intern...|
+--------------------+--------------------+
only showing top 2 rows



In [30]:
# create dataframe that contains airports matched and unmatched reuslt from the join
# Rename columns in result_df
result_df = result_df.withColumnRenamed("name", "weather_matched") \
                     .withColumnRenamed("departing_airport", "delay_matched")

# Rename columns in enhanced_result_df
enhanced_result_df = enhanced_result_df.withColumnRenamed("name", "weather_unmatched") \
                                       .withColumnRenamed("departing_airport", "delay_unmatched")

# Optional: If you need to ensure the rows are matched by order, add an index column to each DataFrame
result_df = result_df.withColumn("index", monotonically_increasing_id())
enhanced_result_df = enhanced_result_df.withColumn("index", monotonically_increasing_id())

# Join DataFrames on the index column
matched_and_unmatched_airports = result_df.join(
    enhanced_result_df,
    on="index",
    how="outer"  # Use "outer" to include all rows from both DataFrames
)

# Drop the index column as it's no longer needed after joining
matched_and_unmatched_airports = matched_and_unmatched_airports.drop("index", 'name', 'departing_airport')

# Show the resulting DataFrame structure
matched_and_unmatched_airports.drop('id')
matched_and_unmatched_airports.show(n = 2)


+--------------------+--------------------+---+--------------------+--------------------+
|       delay_matched|     weather_matched| id|   weather_unmatched|     delay_unmatched|
+--------------------+--------------------+---+--------------------+--------------------+
|memphis internati...|memphis internati...|  0|albany internatio...|tucson international|
|portland internat...|portland internat...|  1|albuquerque inter...|charleston intern...|
+--------------------+--------------------+---+--------------------+--------------------+
only showing top 2 rows



In [29]:
#count the amount of airports for each clumn in the enhanced dataset dataframe 

non_null_name_count = result_df.filter(col("name").isNotNull()).count()
non_null_name_count1 = result_df.filter(col("departing_airport").isNotNull()).count()
non_null_name_count2 = enhanced_result_df.filter(col("weather_unmatched").isNotNull()).count()
non_null_name_count3 = enhanced_result_df.filter(col("delay_unmatched").isNotNull()).count()
print("Number of non-null strings in the 'name' column:", non_null_name_count, non_null_name_count1,non_null_name_count2, non_null_name_count3)

# Display the filtered DataFrame and print the counts



Number of non-null strings in the 'name' column: 30 30 76 56


In [31]:
import pandas as pd

# Initialize a list to store the parsed data
data = []

# Open the text file and parse it line by line
with open('./airports.txt', 'r') as file:
    for line in file:
        # Split the line by comma to extract the needed parts
        parts = line.split(',')
        
        # Check if the line has enough parts to avoid index errors
        if len(parts) >= 4:
            # Extract and clean the desired parts
            # Remove quotation marks and extra spaces if present
            name = parts[1].strip('"').strip()
            city = parts[2].strip('"').strip()
            country = parts[3].strip('"').strip()
            
            # Combine the first two parts into one column, and keep the country as the second column
            combined = f"{name}, {city}, {country}"
            data.append(combined)

# Create a DataFrame from the list
df_airports = pd.DataFrame(data, columns=['Airport and City'])

# Display the DataFrame to verify it's correct
print(df_airports)


                                       Airport and City
0              Goroka Airport, Goroka, Papua New Guinea
1              Madang Airport, Madang, Papua New Guinea
2     Mount Hagen Kagamuga Airport, Mount Hagen, Pap...
3              Nadzab Airport, Nadzab, Papua New Guinea
4     Port Moresby Jacksons International Airport, P...
...                                                 ...
7693                Rogachyovo Air Base, Belaya, Russia
7694            Ulan-Ude East Airport, Ulan Ude, Russia
7695             Krechevitsy Air Base, Novgorod, Russia
7696        Desierto de Atacama Airport, Copiapo, Chile
7697             Melitopol Air Base, Melitopol, Ukraine

[7698 rows x 1 columns]


In [32]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

from fuzzywuzzy import process, fuzz

def get_matches(df1, col1, df2, col2, threshold=40):
    # Convert each column to a list for processing, ensuring to drop NA values
    list1 = df1[col1].dropna().tolist()
    list2 = df2[col2].dropna().tolist()

    # Find best matches with a score above the threshold
    matches = []
    for item in list1:
        # Use process.extractOne to find the best match for each item from list1 in list2
        best_match = process.extractOne(item, list2, scorer=fuzz.token_set_ratio)
        if best_match and best_match[1] >= threshold:
            matches.append((item, best_match[0], best_match[1]))

    # Return matches as a DataFrame for better visualization
    return pd.DataFrame(matches, columns=[col1, col2 + '_match', 'Score'])

# Assuming 'matched_and_unmatched_airports' is your PySpark DataFrame
pandas_df = matched_and_unmatched_airports.toPandas()  # Convert to Pandas DataFrame

# Example usage (ensure df1 and df2 are already defined and loaded with your data)
df_matches_weather = get_matches(pandas_df, 'weather_unmatched', df_airports, 'Airport and City')
df_matches_delay = get_matches(pandas_df, 'delay_unmatched', df_airports, 'Airport and City')


# Merge the two match DataFrames on the 'Airport and City' match column
combined_matches = pd.merge(
    df_matches_weather,
    df_matches_delay,
    on='Airport and City_match',
    suffixes=('_weather', '_delay')
)

# Select the relevant columns
final_matches = combined_matches[['weather_unmatched', 'delay_unmatched', 'Airport and City_match']]

# Rename 'Airport and City_match' for clarity
final_matches.rename(columns={'Airport and City_match': 'Airport and City'})


print(final_matches)

spark_df = spark.createDataFrame(final_matches)

# Show the DataFrame to verify conversion
spark_df.show()


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


                                    weather_unmatched  \
0   anchorage ted stevens international airport, a...   
1       austin bergstrom international airport, tx us   
2                                       boston, ma us   
3                     charleston intl. airport, sc us   
4      cleveland hopkins international airport, oh us   
5                           dallas faa airport, tx us   
6             des moines international airport, ia us   
7        fort lauderdale international airport, fl us   
8                           greensboro airport, nc us   
9           indianapolis international airport, in us   
10                           knoxville airport, tn us   
11                  milwaukee mitchell airport, wi us   
12  minneapolis st. paul international airport, mn us   
13                         new orleans airport, la us   
14                             phoenix airport, az us   
15                             raleigh airport, nc us   
16                             

In [ ]:
print(joined_df.count())
print(df_flightdelay.count())

In [ ]:


fractions = {label: 0.1 for label in joined_df.select("DEP_DEL15").distinct().rdd.flatMap(lambda x: x).collect()}
sampled_df = joined_df.stat.sampleBy("DEP_DEL15", fractions, seed=1234)

# Show the sampled data distribution
sampled_df.groupBy("DEP_DEL15").count().show()

# Split the DataFrame into training (60%) and test (40%) sets
train_df, test_df = sampled_df.randomSplit([0.6, 0.4], seed=1234)

# Show the size of each set
print("Training Dataset Count: " + str(train_df.count()))
print("Testing Dataset Count: " + str(test_df.count()))


In [ ]:
#numerical to nominal
# Calculate the quantile thresholds
thresholds = joined_df.approxQuantile("PRCP", [0.33, 0.67], 0.01)  # 0.01 is the relative error

# Categorize based on quantile thresholds
joined_df = joined_df.withColumn(
    "precip_category",
    when(col("PRCP") <= thresholds[0], "low")
    .when(col("PRCP") <= thresholds[1], "medium")
    .otherwise("high")
)

# Show the resulting DataFrame
joined_df.select("PRCP", "precip_category").show()

In [ ]:
grouped_dfi_lower = grouped_df.select(col("NAME").alias("name").cast("string"))

# Select columns from df_flightdelay and rename to lowercase
df_flightdelay_lower = df_flightdelay.select(col("DEPARTING_AIRPORT").alias("departing_airport").cast("string"))

# Join the two DataFrames on some common column, for example, index
result_df = grouped_dfi_lower.join(df_flightdelay_lower, grouped_dfi_lower.index == df_flightdelay_lower.index, "inner").drop(df_flightdelay_lower.index)
show(result_df)
